In [1]:
import os
import datetime as dt
import sys
import json
import traceback
import logging
import numpy as np
from googleapiclient.discovery import build
from google.oauth2  import service_account
import httplib2
import google_auth_httplib2
from pprint import pprint
from PIL import Image
from ImageToJSON import RequestBuilder

In [2]:
imagepath = ""
sheetName = ""
sheetID = 0
SQUAREWIDTH = 21
SPREADSHEET_ID = '1KwBAK_ab6fhbt_Cq3RyOfHtGHICyFqhZC2dveGCpAKs'
SERVICE_KEY = 'key.json'

In [3]:
imagepath = "images\monasmall.png"

inputimage = Image.open(imagepath)

width, height  = inputimage.size
print('Image loaded. Width = ', width, ', Height = ', height)

#Checking if image mode is RGB and convert to RGB if required
print("Image mode = " + inputimage.mode)

#print(inputimage.getbands())

if inputimage.mode != "RGB":
    print("Image mode is not RGB. Converting to RGB")
    inputimage = inputimage.convert("RGB")

#Loading image to the pixel data. This is an internal data structure of Pillow
image_sequence = inputimage.getdata()

#Sequence to np array for easy handling
image_array = np.array(image_sequence)

Image loaded. Width =  200 , Height =  200
Image mode = P
Image mode is not RGB. Converting to RGB


In [4]:
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

creds = None

creds = service_account.Credentials.from_service_account_file(SERVICE_KEY, 
                                                              scopes=SCOPES)

service = build('sheets', 'v4', credentials=creds)

In [5]:
#Simple random sheet name generator
#Month WeekdayName interger part of timestamp
now = dt.datetime.now()
sheetName = f"{now:%b} {now:%a} {int(now.timestamp())}" 
print(f"Sheet Name: {sheetName}")

#Creating as new Sheet in the SpreadSheet. 
# Using the Image dimensions as the Sheet dimensions.
requests = []

requests.append(
    {
        "addSheet": {
            "properties": {
                "title": sheetName,
                "gridProperties": {
                    "rowCount": height,
                    "columnCount": width
                }
            }
        }
    }            
)

body = {
    'requests': requests
}

print("Adding new Sheet")

addsheetresponse = service.spreadsheets().batchUpdate(
                            spreadsheetId=SPREADSHEET_ID, 
                            body=body).execute()
print("Added new Sheet")

Sheet Name: Sep Tue 1695734834
Adding new Sheet
Added new Sheet


In [6]:
#Changing the column width of all the Columns to make it square

sheetID = addsheetresponse['replies'][0]['addSheet']['properties']['sheetId']

requests = []

requests.append({
    'updateDimensionProperties': {
        'range': {
            'sheetId': sheetID,
            'dimension': "COLUMNS",
            'startIndex': 0,
            'endIndex': width,
        },
        'properties': {
            'pixelSize': SQUAREWIDTH
        },
        'fields': 'pixelSize'
    }
})

body = {
    'requests': requests
}

print("Setting square cells")

setwidthresponse = service.spreadsheets().batchUpdate(
                            spreadsheetId=SPREADSHEET_ID, 
                            body=body).execute()

#pprint(setwidthresponse)

print("Setup of square cell is complete")

Setting square cells
Setup of square cell is complete


In [7]:
#Generating the JSON using the Image data

print("Building the request JSON from image pixel data")

requestSON = RequestBuilder.ImageArrayToJSON(sheetID, image_array, 
                                             width, height)

print("JSON generation completed")
#print(requestSON)

logging.info(requestSON)

#We need build JSON object from string as API takes JSON object
request = json.loads(requestSON)

Building the request JSON from image pixel data
Array dimension = {} (40000, 3)
JSON generation completed


In [8]:
print("Executing batch request")

response = service.spreadsheets().batchUpdate(spreadsheetId=SPREADSHEET_ID,
                                              body=request).execute()

print("Executed batch update")

Executing batch request
Executed batch update
